# T4 - Filtering & time series
Before we look at the full (multivariate) Kalman filter,
let's get more familiar with time-dependent (temporal/sequential) problems.
$
% START OF MACRO DEF
% DO NOT EDIT IN INDIVIDUAL NOTEBOOKS, BUT IN macros.py
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}}
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}}
%
\newcommand{\trsign}{{\mathsf{T}}}
\newcommand{\tr}{^{\trsign}}
\newcommand{\tn}[1]{#1}
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}}
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}}
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$

In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
plt.ion();

In [ ]:
# Use ObsMod=1 so that it makes sense to plot data on same axes as state.
ObsMod = 1

xa = 0  # initial estimate mean
Pa = 100  # initial estimate variance

def simulate(K, xa, Pa, DynMod, ObsMod, Q, R):
    """Simulate synthetic truth (x) and observations (y)."""
    x = xa + np.sqrt(Pa)*rnd.randn()            # Draw initial condition
    truths = np.zeros(K)                        # Allocate
    obsrvs = np.zeros(K)                        # Allocate
    for k in range(K):                          # Loop in time
        x = DynMod * x + np.sqrt(Q)*rnd.randn() # Dynamics
        y = ObsMod * x + np.sqrt(R)*rnd.randn() # Measurement
        truths[k] = x                           # Assign
        obsrvs[k] = y                           # Assign
    return truths, obsrvs

In [ ]:
r = dict(justify_content='flex-end')
@ws.interact(seed=(1, 9), K=(0, 100), DynMod=(-1.03, 1.03, .01),
             logR=(-8, 8), logR_bias=(-8, 8),
             logQ=(-8, 8), logQ_bias=(-8, 8))
def plot_experiment(seed, K, DynMod=0.97, logR=1, logQ=1, analyses_only=False,
                    logR_bias=0, logQ_bias=0):
    rnd.seed(seed)
    R      = np.exp(logR)
    Q      = np.exp(logQ)
    Q_bias = np.exp(logQ_bias)
    R_bias = np.exp(logR_bias)

    truths, obsrvs = simulate(K, xa, Pa, DynMod, ObsMod, Q, R)

    kk = 1 + np.arange(K)
    plt.figure(figsize=(9, 6))
    plt.plot(kk, truths, 'k' , label='True state ($x$)')
    plt.plot(kk, obsrvs, 'g*', label='Noisy obs ($y$)')

    try:
        estimates, variances = KF(K, xa, Pa, DynMod, ObsMod, Q*Q_bias, R*R_bias, obsrvs)
        # +/- 1-sigma (std.dev.) credible intervals (CI):
        upper = estimates + np.sqrt(variances)
        lower = estimates - np.sqrt(variances)
        if analyses_only:
            plt.plot(kk, estimates[:, 1], label='Kalman$^a$ ± 1$\sigma$')
            plt.fill_between(kk, lower[:, 1], upper[:, 1], alpha=.2)
        else:
            kk = kk.repeat(2)
            plt.plot(kk, estimates.flatten(), label='Kalman ± 1$\sigma$')
            plt.fill_between(kk, lower.flatten(), upper.flatten(), alpha=.2)
    except NameError:
        pass

    plt.xlabel('Time index (k)')
    plt.grid()
    plt.legend(loc='upper left')
    plt.show()
    print(variances)

Answer the following in order.
- What does `seed` control?
- Explain what happens when `DynMod =0, >1, <-1`.
- What happens when $R \rightarrow 0$ ?
- What happens when $R \rightarrow \infty$ ?

In [ ]:
def KF(K, xa, Pa, DynMod, ObsMod, Q, R, obsrvs):
    estimates = np.zeros((K, 2))
    variances = np.zeros((K, 2))
    for k in range(K):
        # Forecast step
        xf = DynMod * xa
        Pf = DynMod**2 * Pa + Q
        # Analysis update step
        Pa = 1 / (1/Pf + ObsMod**2/R)
        xa = Pa * (xf/Pf + ObsMod*obsrvs[k]/R)
        # Assign
        estimates[k] = xf, xa
        variances[k] = Pf, Pa
    return estimates, variances

Re-run the above interative animation.

- `logR_bias` controls the (multiplicative) bias in $R$ that is fed to the KF.
  What happens when the KF thinks the measurement error is much smaller
  than it actually is? What about when it's much larger?
  **Finally, reset `logR_bias=0`.**
- Similarly, `logQ_bias` controls the bias in $Q$, i.e. the (dynamical) model error.
  Answer the same questions.
  **Finally, reset `logQ_bias=0`.**
- Maximize/minimize `logR_bias`/`logQ_bias`, respectively.
  The uncertainty should shrink in time, looking like a funnel.
  Explain this.
- Maximize `logR_bias`. Set `logQ_bias=2`. Explain what you see.
- TODO: make the shrinking of P more obvious. Also under non-extreme circumstances.

- Add `100` to the initial `xa` given to the KF.
  How long does it take for it to recover from this initial bias?
- Multiply `Pa` by `0.001`. What about now?